In [71]:
import seekpath
from seekpath.brillouinzone.brillouinzone import get_BZ

import ase.io
import ase.visualize

import numpy as np

from pathlib import Path

In [85]:
struct_file = Path("./mc3d-10000.cif")
atoms = ase.io.read(struct_file)

In [86]:
seekpath_structure = (atoms.cell, atoms.get_scaled_positions(), atoms.numbers)

In [87]:
def get_json_for_visualizer(cell, relcoords, atomic_numbers):
    # Implementation from tools-seekpath
    # https://github.com/materialscloud-org/tools-seekpath/blob/3d5acaa627d075a3c9ef9d728f9e4fc9824f7cd3/compute/seekpath_web_module.py#L36
    hpkot = seekpath.hpkot
    brillouinzone = seekpath.brillouinzone

    system = (np.array(cell), np.array(relcoords), np.array(atomic_numbers))
    res = hpkot.get_path(system, with_time_reversal=False)

    real_lattice = res["primitive_lattice"]
    # rec_lattice = np.linalg.inv(real_lattice).T # Missing 2pi!
    rec_lattice = np.array(hpkot.tools.get_reciprocal_cell_rows(real_lattice))
    b1, b2, b3 = rec_lattice

    faces_data = brillouinzone.brillouinzone.get_BZ(b1=b1, b2=b2, b3=b3)

    response = {}
    response["faces_data"] = faces_data
    response["b1"] = b1.tolist()
    response["b2"] = b2.tolist()
    response["b3"] = b3.tolist()
    ## Convert to absolute
    response["kpoints"] = {
        k: (v[0] * b1 + v[1] * b2 + v[2] * b3).tolist()
        for k, v in res["point_coords"].items()
    }
    response["kpoints_rel"] = {
        k: [v[0], v[1], v[2]] for k, v in res["point_coords"].items()
    }
    response["path"] = res["path"]

    # It should use the same logic, so give the same cell as above
    res_explicit = seekpath.get_explicit_k_path(system, with_time_reversal=False)
    for k in res_explicit:
        if k == "segments" or k.startswith("explicit_"):
            if isinstance(res_explicit[k], np.ndarray):
                response[k] = res_explicit[k].tolist()
            else:
                response[k] = res_explicit[k]

    if (
        np.sum(
            np.abs(
                np.array(res_explicit["reciprocal_primitive_lattice"])
                - np.array(res["reciprocal_primitive_lattice"])
            )
        )
        > 1.0e-7
    ):
        raise AssertionError("Got different reciprocal cells...")

    # Response for JS, and path_results
    return response, res


In [88]:
response, res = get_json_for_visualizer(atoms.cell, atoms.get_scaled_positions(), atoms.numbers)
response.keys()

dict_keys(['faces_data', 'b1', 'b2', 'b3', 'kpoints', 'kpoints_rel', 'path', 'explicit_kpoints_rel', 'explicit_kpoints_linearcoord', 'explicit_kpoints_labels', 'explicit_kpoints_abs', 'explicit_segments'])

In [89]:
def get_data_for_visualizer(cell, relcoords, atomic_numbers):
    # new implementation
    system = (np.array(cell), np.array(relcoords), np.array(atomic_numbers))
    res = seekpath.get_explicit_k_path(system, with_time_reversal=False)
    
    b1, b2, b3 = res['reciprocal_primitive_lattice']
    faces_data = get_BZ(b1=b1, b2=b2, b3=b3)

    kpoints_rel = res["point_coords"]
    kpoints_abs = {
        k: (v[0] * np.array(b1) + v[1] * np.array(b2) + v[2] * np.array(b3)).tolist()
        for k, v in kpoints_rel.items()
    }

    response = {
        "faces_data": faces_data,
        "b1": b1,
        "b2": b2,
        "b3": b3,
        "kpoints": kpoints_abs,
        "kpoints_rel": kpoints_rel,
        "path": res["path"],
        "explicit_kpoints_rel": res["explicit_kpoints_rel"].tolist(),
        "explicit_kpoints_linearcoord": res["explicit_kpoints_linearcoord"].tolist(),
        "explicit_kpoints_labels": res["explicit_kpoints_labels"],
        "explicit_kpoints_abs": res["explicit_kpoints_abs"].tolist(),
        "explicit_segments": res["explicit_segments"],
    }

    return response

In [90]:
res_new = get_data_for_visualizer(atoms.cell, atoms.get_scaled_positions(), atoms.numbers)

In [91]:
# check if the new function gives the same output as the old one
res_new == response

True

In [92]:
import json

with open(f"./{struct_file.stem}.json", "w") as f:
    json.dump(res_new, f, indent=2)